## Google Sentence Piece를 이용해서 Vocab 파일을 만드는 과정
Google SentencePeice와 한국어 위키를 이용해서 Vocab을 만드는 과정에 대한 설명 입니다.

[Colab](https://colab.research.google.com/)에서 실행 했습니다.

#### 0. Pip Install
필요한 패키지를 pip를 이용해서 설치합니다.

In [0]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 4.8MB/s 


#### 1. Google Drive Mount
Colab에서는 컴퓨터에 자원에 접근이 불가능 하므로 Google Drive에 파일을 올려 놓은 후 Google Drive를 mount 에서 로컬 디스크처럼 사용 합니다.
1. 아래 블럭을 실행하면 나타나는 링크를 클릭하세요.
2. Google 계정을 선택 하시고 허용을 누르면 나타나는 코드를 복사하여 아래 박스에 입력한 후 Enter 키를 입력하면 됩니다.

In [0]:
from google.colab import drive
drive.mount('/content/drive')
# data를 저장할 폴더 입니다. 환경에 맞게 수정 하세요.
data_dir = "/content/drive/My Drive/Data/transformer-evolution"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 2. Imports


In [0]:
import os
import gzip
import pandas as pd
import sentencepiece as spm
import shutil

#### 3. 폴더의 목록을 확인
Google Drive mount가 잘 되었는지 확인하기 위해 data_dir 목록을 확인 합니다.

In [0]:
for f in os.listdir(data_dir):
  print(f)

kowiki.csv.gz


#### 4. CSV 파일을 TEXT 형태로 변환
Google SentencePiece에서 사용할 수 있도록 CSV를 TEXT형태로 변환 합니다.

In [0]:
in_file = f"{data_dir}/kowiki.csv.gz"
out_file = "kowiki.txt"
SEPARATOR = u"\u241D"

df = pd.read_csv(in_file, sep=SEPARATOR, engine="python", compression='gzip')
with open(out_file, "w") as f:
  for index, row in df.iterrows():
    f.write(row["text"]) # title 과 text를 중복 되므로 text만 저장 함
    f.write("\n\n\n\n") # 구분자

#### 5. Vocab 생성
Google SentencePiece를 이용해서 vocab 만드는 방법 입니다. vocab을 만드는데 많은 시간이 소요 됩니다. 잠시 여유를 갖고 기다려 주세요.

In [0]:
corpus = "kowiki.txt"
prefix = "kowiki"
vocab_size = 32000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 기타 추가 토큰

for f in os.listdir("."):
  print(f)
shutil.copy("kowiki.model", f"{data_dir}/kowiki.model")
shutil.copy("kowiki.vocab", f"{data_dir}/kowiki.vocab")

.config
kowiki.model
drive
kowiki.txt
kowiki.vocab
sample_data


'/content/drive/My Drive/Data/transformer-evolution/kowiki.vocab'

#### 6. Vocab 테스트
생성된 vocab을 간단하게 테스트 해 봅니다.

In [0]:
vocab_file = f"{data_dir}/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]
for line in lines:
  pieces = vocab.encode_as_pieces(line)
  ids = vocab.encode_as_ids(line)
  print(line)
  print(pieces)
  print(ids)
  print()

겨울이 되어서 날씨가 무척 추워요.
['▁겨울', '이', '▁되어서', '▁날', '씨가', '▁무척', '▁추', '워', '요', '.']
[3091, 27588, 19397, 683, 5019, 14900, 206, 27958, 27760, 27590]

이번 성탄절은 화이트 크리스마스가 될까요?
['▁이번', '▁성', '탄', '절', '은', '▁화이트', '▁크리스', '마', '스가', '▁될', '까', '요', '?']
[3224, 86, 27967, 27923, 27604, 6340, 1970, 27664, 780, 1450, 27794, 27760, 28245]

겨울에 감기 조심하시고 행복한 연말 되세요.
['▁겨울에', '▁감', '기', '▁조심', '하시', '고', '▁행복한', '▁연말', '▁되', '세요', '.']
[18838, 212, 27605, 20179, 5871, 27600, 22057, 19628, 445, 16682, 27590]

